In [1]:
#Importing useful packages
import sqlite3
import csv
import pandas as pd
from datetime import datetime

#Returns a Dataframe containing a sample of size "number" from the specified month and year
def get_sample(year,month,number):
    #Change to directory of your database!
    conn = sqlite3.connect(r'E:\Summer Project\yellow_taxi_'+str(year)+'.sqlite')
    c = conn.cursor()
    sample = pd.read_sql_query('select * from '+str(month)+' where rowid = (abs(random()) % (select (select max(rowid) from january)+1))',conn)
    for i in range(number):
        sample = sample.append(pd.read_sql_query('select rowid, * from '+str(month)+' where rowid = (abs(random()) % (select (select max(rowid) from january)+1))',conn))#row,ignore_index=True)
    return sample

In [2]:
#Example of get_sample
x = get_sample(2013,"may",20)

In [3]:
print(x)

      dropoff_datetime  dropoff_latitude  dropoff_longitude  fare_amount  \
0  2013-05-21 06:49:14         40.721658         -74.008193          8.0   
0  2013-05-27 01:24:00         40.742650         -74.004240          9.5   
0  2013-05-17 08:48:00         40.771212         -73.988160          4.5   
0  2013-05-01 12:10:00         40.753997         -73.972147         26.5   
0  2013-05-11 21:13:00         40.750197         -73.990337         11.5   
0  2013-05-16 02:12:33         40.740218         -74.007984          8.0   
0  2013-05-21 10:35:00         40.752832         -73.965737          7.0   
0  2013-05-18 11:42:58         40.705169         -74.016824         22.0   
0  2013-05-01 12:39:32         40.735577         -73.992507          6.5   
0  2013-05-19 21:18:00         40.763545         -73.954695         10.0   
0  2013-05-05 10:17:43         40.765119         -73.972466          6.5   
0  2013-05-31 17:51:00         40.675352         -73.999277         43.0   
0  2013-05-0